In [1]:
import requests
import dill
from bs4 import BeautifulSoup
from datetime import datetime
import simplejson as json
import pandas as pd
import numpy as np
from urllib.parse import quote
from urllib.request import urlopen
import spotipy
import time
import re

In [2]:
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = spotipy.oauth2.SpotifyClientCredentials('d6967ce2057448d4aab3ad9898119c97',  'ad7f82cc26a64f1595b6b3c4cd917243')
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
import dill
model = dill.load(open('model.pkd', 'rb'))

In [12]:
import heapq
def hit_song_predictor(album, model):
    X, y = album.train_X_y()
    y_est = model.predict(X)
    indices = np.arange(album.total_tracks)[y_est]
    hit_list = album.tracks_df[[ 'track_number',  'name']].values[indices]

    y_est_prob_mtx = model.predict_proba(X)
    y_est_prob_mtx[:, 0] = np.arange(album.total_tracks)
    indices = np.array(heapq.nlargest(3, y_est_prob_mtx, key=lambda x: x[1]))[:,0].astype(int)
    
    top_three = album.tracks_df[[ 'track_number',  'name']].values[indices] 
        
    return hit_list, top_three

In [5]:
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler

class Album(object):
    def __init__(self, album_json):
        self.id, self.name, self.genres, self.popularity, self.total_tracks, self.artists_list = [album_json[k] for k in ['id', 'name', 'genres', 'popularity', 'total_tracks', 'artists_list']]
        self.tracks_df = pd.read_json(album_json['tracks_info'], orient='split')

    def unit_transf(self):
        self.tracks_df['tempo'] = self.tracks_df['tempo'] / 60
        self.tracks_df['duration_ms'] = self.tracks_df['duration_ms'] / 1000 / 60
        self.tracks_df['loudness'] = self.tracks_df['loudness'] / 10
        self.tracks_df['ordering'] = MinMaxScaler().fit_transform(self.tracks_df['track_number'].values.reshape(-1,1)) - 0.5
        self.tracks_df['total_tracks'] = self.total_tracks
        
    def classification_label(self):
        self.tracks_df['label'] = self.tracks_df['popularity'] >= self.popularity
        return self.tracks_df['label'].values
        
    def train_X_y(self):
        self.unit_transf()        
        X = self.tracks_df[['mode', 'tempo', 'duration_ms', 'ordering', 'acousticness', 'danceability', 'energy', 'liveness', 'speechiness','valence']].values
        y = self.classification_label()
        return X, y

In [6]:
def strip_Feat(artists):
    pos = artists.find('Featuring')
    if pos == -1:
        return artists
    else:
        return artists[:pos].strip()
    
def get_track_artists(year, domain = 'https://www.billboard.com/charts/year-end/'):
    path = domain + str(year) + '/hot-100-songs'
    response = requests.get(path)
    soup = BeautifulSoup(response.text, "lxml")

    parents = soup.findAll(class_=r'ye-chart-item__text')

    track_artists = []
    for parent in parents:
        track = parent.find('div',attrs={'class':'ye-chart-item__title'}).get_text().strip()
        artists = parent.find('div',attrs={'class':'ye-chart-item__artist'}).get_text().strip()
        track_artists.append((track, artists))
    return track_artists


def get_album_artists(year, domain='https://www.billboard.com/archive/charts/'):
    response = requests.get(domain + str(year) + "/top-album-sales")
    soup = BeautifulSoup(response.text, "lxml")
    table = soup.find('table',attrs={'class':'archive-table'}).findAll('tr')[1:]

    album_artists=[]
    for tr in table:
        td = tr.findAll('td')
        try:
            album_artists.append((td[-2].get_text(),td[-1].get_text()))
        except:
            pass
    return album_artists

def get_track_artists_arxiv(year, domain='https://www.billboard.com/archive/charts/'):
    path = domain + str(year) + '/hot-100'
    response = requests.get(path)
    soup = BeautifulSoup(response.text, "lxml")

    table = soup.find('table',attrs={'class':'archive-table'}).findAll('tr')[1:]

    track_artists=[]
    for tr in table:
        td = tr.findAll('td')
        try:
            track_artists.append((td[-2].get_text(),td[-1].get_text()))
        except:
            pass
    return track_artists

def get_album_id_from_track(track, artists):
    try:
        track = track.replace("'",'')
        artists = ", ".join(re.split(r' & | x | X | With | with ', artists))
        q = 'track:' + track + ' artist:' + artists
        result = spotify.search(q, limit=1)
        the_first_album = result['tracks']['items'][0]['album']
        album_id = the_first_album.get('id')
        return album_id
    except:
#         try:
#             artists = ' '.join(artists.split(' x '))
#             artists = ' '.join(artists.split(' X '))
#             q = track + ' ' + artists
#             result = spotify.search(q, limit=1)
#             the_first_album = result['tracks']['items'][0]['album']
#             album_id = the_first_album.get('id')
#             return album_id
#         except:
        return track, artists

def get_album_id_from_album(album, artists):

    try:
        album = album.replace("'",'')        
        artists = ", ".join(re.split(r' & | x | X | With | with ', artists))
        q = 'album:' + album + ' artist:' + artists
        result = spotify.search(q, type='album', limit=1)
        the_first_album = result['albums']['items'][0]
        album_id = the_first_album.get('id')
        return album_id
    
    except:
#         try:
#             q =  album
#             result = spotify.search(q, type='album', limit=1)
#             the_first_album = result['albums']['items'][0]
#             album_id = the_first_album.get('id')
#             return album_id
#         except:
        return album, artists

def get_albums_id_list(year, get_album_id_func, scraping_func):
    albums_id = []    


    trackOrAlbum_artists = scraping_func(year)
    for trackOrAlbum, artists in trackOrAlbum_artists:
        albums_id.append(get_album_id_func(trackOrAlbum,strip_Feat(artists)))
        
    return albums_id

def album_json(album_id):

    d = spotify.album(album_id)
    subkeys = ['id', 'name', 'genres',  'popularity', 'total_tracks']

    subkeys = ['id', 'name', 'genres',  'popularity', 'total_tracks']
    
    album_info = {k: d[k] for k in subkeys if k in d}
    tracks = d['tracks']['items']
    tracks_id = [t['id'] for t in tracks]
    tracks_df_json = pd.DataFrame(audio_features(tracks_id),columns=['id',  'track_number', 'popularity','name', 'duration_ms', 'tempo','time_signature', 'key',
       'valence', 'mode', 'acousticness', 'danceability', 'energy', 
       'instrumentalness',  'liveness', 'loudness', 'speechiness']).to_json(orient='split')
    album_info.update({'artists_list': [a['name'] for a in d['artists']], 'tracks_info': tracks_df_json })
    
    return album_info

def track_json(track_id):
    d = spotify.track(track_id)
    subkeys = ['id', 'popularity', 'name', 'track_number']
    subdict = {k: d[k] for k in subkeys if k in d}
    return subdict


def audio_feature_decorator(spotify_audio_features_func):

    def wrapper_func(track_id_list):
        subkeys = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
        
        
        features_list = spotify_audio_features_func(track_id_list)
        
        my_features_list = []
        for i, features in enumerate(features_list):
            sub_features = {k: features[k] for k in subkeys if k in features}
            other_info = track_json(track_id_list[i])
            other_info.update(sub_features)
            
            my_features_list.append(other_info)
            
        return my_features_list
    
    return wrapper_func

@audio_feature_decorator
def audio_features(track_id_list):
    return spotify.audio_features(track_id_list)


In [55]:
album_id = get_album_id_from_album('Ricky Sings Again', 'Ricky Nelson')

In [56]:
abj = album_json(album_id)
a = Album(abj)

In [57]:
X, y = a.train_X_y()

In [13]:
hit_ls, top_three = hit_song_predictor(a, model)

In [40]:
def track_print(ls):
    string = ""
    for n, t in ls:
        string += (str(n) + '. ' + t ) + '\n'
    
    return string[:-2] 

In [41]:
track_print(hit_ls)

''

In [53]:
track_print(top_three)

'3. Believe What You Say - Remastered\n7. Old Enough To Love - Remastered\n2. One Of These Mornings - Remastere'

In [60]:
import seaborn as sns

cm = sns.light_palette("pink", as_cmap=True)


a.tracks_df[['track_number', 'acousticness','danceability','energy','liveness']].set_index('track_number').style.background_gradient(cmap=cm)

,acousticness,danceability,energy,liveness
track_number,,,,
1,0.787,0.614,0.789,0.0938
2,0.441,0.715,0.626,0.0737
3,0.677,0.663,0.796,0.0817
4,0.806,0.509,0.184,0.103
5,0.648,0.646,0.252,0.318
6,0.778,0.46,0.305,0.304
7,0.681,0.693,0.488,0.101
8,0.127,0.58,0.445,0.136
9,0.493,0.703,0.447,0.376


In [61]:
a.tracks_df[['track_number', 'acousticness','danceability','energy','liveness']].set_index('track_number').style.background_gradient(cmap=cm)._repr_html_()

'<style  type="text/css" >\n    #T_519ca64a_87f9_11e9_844d_8438355d3b84row0_col0 {\n            background-color:  #ffc3ce;\n        }    #T_519ca64a_87f9_11e9_844d_8438355d3b84row0_col1 {\n            background-color:  #ffd4dc;\n        }    #T_519ca64a_87f9_11e9_844d_8438355d3b84row0_col2 {\n            background-color:  #ffc0cb;\n        }    #T_519ca64a_87f9_11e9_844d_8438355d3b84row0_col3 {\n            background-color:  #ffe2e7;\n        }    #T_519ca64a_87f9_11e9_844d_8438355d3b84row1_col0 {\n            background-color:  #ffd4db;\n        }    #T_519ca64a_87f9_11e9_844d_8438355d3b84row1_col1 {\n            background-color:  #ffccd5;\n        }    #T_519ca64a_87f9_11e9_844d_8438355d3b84row1_col2 {\n            background-color:  #ffc8d2;\n        }    #T_519ca64a_87f9_11e9_844d_8438355d3b84row1_col3 {\n            background-color:  #ffe4e9;\n        }    #T_519ca64a_87f9_11e9_844d_8438355d3b84row2_col0 {\n            background-color:  #ffc8d2;\n        }    #T_519ca64a_87

In [71]:
pd.DataFrame(top_three).to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>3</td>\n      <td>Believe What You Say - Remastered</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>7</td>\n      <td>Old Enough To Love - Remastered</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>2</td>\n      <td>One Of These Mornings - Remastered</td>\n    </tr>\n  </tbody>\n</table>'

In [72]:
pd.DataFrame(top_three).to_html(index=False, header=False)

'<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>3</td>\n      <td>Believe What You Say - Remastered</td>\n    </tr>\n    <tr>\n      <td>7</td>\n      <td>Old Enough To Love - Remastered</td>\n    </tr>\n    <tr>\n      <td>2</td>\n      <td>One Of These Mornings - Remastered</td>\n    </tr>\n  </tbody>\n</table>'

In [70]:
top_three

array([[3, 'Believe What You Say - Remastered'],
       [7, 'Old Enough To Love - Remastered'],
       [2, 'One Of These Mornings - Remastered']], dtype=object)

In [73]:
pd.DataFrame([]).to_html(index=False, header=False)

'<table border="1" class="dataframe">\n  <tbody>\n  </tbody>\n</table>'